# 要約 
このノートブックは、LMSYS - Chatbot Arenaコンペティションにおいて、ユーザーの好みを予測するための機械学習モデルのトレーニングおよび推論に取り組んでいます。特に、KerasとTensorFlowを使用して、Gemma Causal Language Modelを基にしたモデルを構築しています。

### 主な問題
ノートブックの目的は、ユーザーからの応答データに基づいて、どのチャットボットの応答が好まれるかを予測するモデルを訓練することです。このモデルは、モデルのトークン化やデータ前処理を行い、各種ハイパーパラメータによる最適化を実施します。

### 使用している手法とライブラリ
1. **KerasとTensorFlow**:
   - Kerasの分散APIを利用して、TPUにモデルをデプロイし、メモリ断片化を防ぐために設定を行います。
   - JAXをバックエンドに使用し、TPU上でのトレーニングと推論を高速化します。

2. **モデル構築**:
   - Gemma Causal Language Modelをロードし、そのバックボーンを利用して、ユーザーの入力とレスポンスを処理するモデルを構築します。
   - 出力層にはソフトマックス関数を適用し、多クラス分類を行います。

3. **データ処理**:
   - CSVファイルからデータを読み込み、必要な列を抽出してサロゲートペアを除去する関数を定義します。
   - TensorFlowのデータセットAPIを使用して、バッチ化と前処理を効率的に行います。

4. **ハイパーパラメータの設定**:
   - AdamWオプティマイザーを使用し、学習率や重み減衰を調整することでモデルのトレーニング性能を向上させます。

5. **モデルのトレーニングと推論**:
   - 訓練データに対してモデルをコンパイルし、予測を行います。最終的な予測結果はCSV形式で保存され、提出用にフォーマットされます。

### コメントと次のステップ
ノートブックでは、タイムアウト問題や推論速度の向上についても言及されており、今後の改善が期待されています。また、他の参加者からのフィードバックや提案も含まれており、相互学習や情報共有の場が形成されています。

全体として、このノートブックは、自然言語処理タスクにおける現代的なアプローチを採用しており、効率的なモデルの構築とデータ処理の手法を示しています。

---


# 用語概説 
以下に、Jupyter Notebookに登場する専門用語の簡単な解説を列挙します。これは、機械学習や深層学習に一定の知識があるが、実務経験が少ない初心者向けです。

1. **float16**: 浮動小数点数を表すデータ型の一つで、16ビットで表現される。計算精度は低いが、メモリ使用量が少なく、ディープラーニングのトレーニングや推論では高速化が期待される。

2. **TPU (Tensor Processing Unit)**: Googleが開発した、ディープラーニング専用のハードウェア加速装置。通常のCPUやGPUに比べて、特にテンソル演算に最適化されているため、モデルのトレーニングや推論を高速化できる。

3. **メモリ断片化 (Memory Fragmentation)**: プログラムがメモリを使用したり解放したりする過程で、メモリ空間が小さな断片に分かれ、十分な連続したメモリを確保できなくなる現象。TPUではメモリの効率的な使用が重要。

4. **デバイスメッシュ (Device Mesh)**: 複数のデバイス（GPUやTPUなど）をグリッド状に配置し、計算を並行して行うための概念。これにより、計算負荷を分散しスピードアップが可能になる。

5. **レイアウトマップ (LayoutMap)**: モデルのレイヤーやパラメータを、デバイスメッシュ上にどのように分散させるかを指定するためのマッピング。

6. **LoRA (Low-Rank Adaptation)**: モデルを少ないパラメータで適応させる手法。特定のタスクに対して重みを効率的に調整できるため、トレーニングコストを削減しつつ性能を向上させることができる。

7. **サロゲートペア (Surrogate Pairs)**: Unicodeにおける特定の文字の範囲（U+D800からU+DFFF）であり、通常の文字コード上では表現できない文字を扱うために使われる。これらのペアを含むテキストを処理する際には特別な対策が必要。

8. **トークナイザー (Tokenizer)**: テキストデータをトークン（単語やサブワードなどの単位）に分割するためのツール。トークンは機械学習モデルが理解できる形式で表現される。

9. **バックボーン (Backbone)**: 特定のタスクに使用されるモデルの基礎部分。通常は特定のアーキテクチャに基づいており、タスクに特化した追加の層がその上に構築される。

10. **トレーニング不可 (Non-trainable)**: 各レイヤーやパラメータがトレーニング中に更新されない設定。この設定は、あらかじめトレーニングされたモデルを再利用する場合に使用される。

理解を助けるために、これらの用語は具体的な文脈において重要であり、それぞれの機能や影響を把握することで、ノートブックのコードの理解が深まります。

---


float16での推論には???時間かかるはずです。

トレーニングノートブック:

https://www.kaggle.com/code/pranshubahadur/tf-gemma-2-9b-lmsys-training-tpu

In [ ]:
import os

# Keras 3の分散APIは、現時点ではJAXバックエンドのみに実装されています
os.environ["KERAS_BACKEND"] = "jax"
# TPUのメモリ断片化と割り当てのオーバーヘッドを最小限に抑えるために、すべてのTPUメモリを事前に割り当てます。
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"
import keras
import keras_nlp

# (1, 2)の形状を持つデバイスメッシュを作成し、すべての8つのTPUに重みを分散させます。
device_mesh = keras.distribution.DeviceMesh(
    (1, 2),  # バッチとモデルのための次元
    ["batch", "model"],
    devices=['gpu:0', 'gpu:1'],  # 使用するGPUデバイスのリスト
)

model_dim = "model"  # モデルの次元を指定

layout_map = keras.distribution.LayoutMap(device_mesh)

# 'token_embedding/embeddings'に一致する重みが8つのTPUに分散されます
layout_map["token_embedding/embeddings"] = (model_dim, None)  # トークン埋め込みのマッピング
layout_map["position_embedding/embeddings"] = (model_dim, None)  # 位置埋め込みのマッピング

# 注意層のクエリ、キー、値行列に対してマッチする正規表現
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)

layout_map["decoder_block.*layer_norm/scale"] = (model_dim,)  # スケールのマッピング
layout_map["decoder_block.*layer_norm/bias"] = (model_dim,)  # バイアスのマッピング

# モデルの分散設定を行います
model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",  # バッチの次元名を指定
)

# モデルの分散をセットします
keras.distribution.set_distribution(model_parallel)

In [ ]:
import jax

# JAXのデフォルトデバイスをCPUに設定します
jax.default_device = jax.devices('cpu')[0]

# 現在のデバイス情報を表示します
jax.devices()  # これにより、使用可能なデバイスのリストが返されます。

In [ ]:
# Kerasのデータ型ポリシーをfloat16に設定します
keras.config.set_dtype_policy("float16")  # これにより、モデルのトレーニングや推論で使用されるデータの型がfloat16になります。

In [ ]:
# このセルにはコードが含まれていません。必要な処理やデータをここに追加してください。

In [ ]:
# Gemma Causal Language Model を事前設定からロードします
# trainable=False でモデルを読み込むと、重みが更新されない設定になります。
# dtype='int8' でデータ型を int8 に設定します。
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma2/keras/gemma2_instruct_9b_en/1", trainable=False, dtype='int8')

# モデルの概要を表示します
gemma_lm.summary()  # これにより、モデルの構造やパラメータの数が表示されます。

In [ ]:
# サロゲートペア（代理対）を削除する関数を定義します
def remove_surrogates(text):
    # サロゲートペアの範囲 (0xD800 から 0xDFFF) に含まれない文字をフィルタリングして結合します
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))  # テキストからサロゲートペアを取り除いた結果を返します。

In [ ]:
from pandas import read_csv, DataFrame

# 使用する入力列の名前を指定
input_columns = ['prompt', 'response_a', 'response_b']
# ラベル列の名前を指定
label_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

# CSVファイルから生のテストデータセットを読み込みます
raw_test_dataset = read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# 以下はコメントアウトされているコードです
# raw_test_dataset[input_columns] = raw_test_dataset[input_columns].map(lambda x: eval(x)[0])  # 各列の中身を評価して最初の要素を取得する
# raw_test_dataset = raw_test_dataset.dropna().reset_index(drop=True)  # NaNを含む行を削除し、インデックスをリセットします

# トレーニングデータセットを作成します
train_dataset = DataFrame({
    # 入力列を結合し、サロゲートペアを削除して新しいカラム 'text' を作成します
    'text' : raw_test_dataset[input_columns].agg('\n\nRESPONSE:\n\n'.join, axis=1)  # 各行の入力を2つのレスポンスで結合
                .apply(lambda x: '\n\nPROMPT\n\n' + x)  # 各行の先頭に極末文を追加
                .apply(lambda x: remove_surrogates(x)),  # サロゲートペアを削除
})

In [ ]:
# Gemmaモデルからトークナイザーとバックボーンを取得します
tokenizer = gemma_lm._preprocessor  # モデルの事前処理（トークナイザー）を取得
backbone = gemma_lm.backbone  # モデルのバックボーン（基盤部分）を取得

In [ ]:
# テキストとオプションのラベルを前処理する関数を定義します
def preprocess_fn(text, label=None):
    # テキストをトークナイザーを使って処理し、トークンIDとパディングマスクを取得します
    preprocessed = tokenizer(text, sequence_length=512)[0]  # シーケンスの長さは512に制限します
    print(preprocessed)  # 前処理された結果を表示します

    # 前処理関数が必要な入力のみを返すことを確認します
    return {'token_ids': preprocessed['token_ids'], 'padding_mask': preprocessed['padding_mask']}  # トークンIDとパディングマスクを含む辞書を返します

In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

# モデルの入力を定義します
inputs = {
        "token_ids": keras.Input(shape=(512,), dtype=tf.int32, name="token_ids"),  # トークンIDの入力
        "padding_mask": keras.Input(shape=(512,), dtype=tf.int32, name="padding_mask"),  # パディングマスクの入力
    }

# バックボーンモデルを通じて入力を処理します
x = backbone(inputs)  # バックボーンに入力を渡します
print(x.shape)  # 出力の形状を表示します

# グローバル平均プーリングを適用します
x = GlobalAveragePooling1D()(x)  # 1次元のデータに対してグローバル平均プーリングを行います
print(x.shape)  # プーリング後の出力の形状を表示します

# 出力層を追加します
outputs = Dense(3, 'softmax')(x)  # 3つのクラスに対してソフトマックス関数を使用する出力層を追加します

# モデルを定義します
model = Model(inputs, outputs)  # 入力と出力からモデルを構築します

In [ ]:
# AdamWオプティマイザーを定義します
optimizer = keras.optimizers.AdamW(
                    learning_rate=5e-5,  # 学習率を5e-5に設定します
                    weight_decay=0.01,  # 重み減衰を0.01に設定します
                )

# バイアスとスケールの変数を重み減衰の対象から除外します
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])  # これにより、指定された変数は重み減衰が適用されません

In [ ]:
# モデルをコンパイルします
model.compile(optimizer,  # 先ほど定義したオプティマイザーを使用
              loss=tf.keras.losses.CategoricalCrossentropy(),  # 多クラスのクロスエントロピー損失を使用
             )  # これにより、指定したオプティマイザーと損失関数でモデルが準備されます

In [ ]:
# モデルの3番目のレイヤーにLoRAの重みをロードします
model.layers[2].load_lora_weights("/kaggle/input/tf-gemma-2-9b-lmsys-training-tpu/model.lora.h5")  # 指定されたファイルからLoRAの重みを読み込みます

In [ ]:
import numpy as np

# 保存された重みとバイアスを読み込みます
dense_1_weights = np.load('/kaggle/input/tf-gemma-2-9b-lmsys-training-tpu/dense_1_kernel.npy')  # 重みを読み込み
dense_1_biases = np.load('/kaggle/input/tf-gemma-2-9b-lmsys-training-tpu/dense_1_bias.npy')  # バイアスを読み込み

# 読み込んだ重みとバイアスを組み合わせてリストを作成します
dense_1_combined = [dense_1_weights, dense_1_biases]

# モデルの最終レイヤーに重みを設定します
model.layers[-1].set_weights(dense_1_combined)  # 最後のレイヤーに対して重みを設定します

In [ ]:
# モデル内のすべてのレイヤーをトレーニング不可に設定します
for layer in model.layers:
    layer.trainable = False  # 各レイヤーのトレーニング可能フラグをFalseに設定し、重みが更新されないようにします

In [ ]:
# モデルの概要を表示します
model.summary()  # モデルのアーキテクチャ、レイヤーの数、パラメータ数などの情報を表示します

In [ ]:
# テンソルスライスからデータセットを作成し、前処理関数を適用してバッチ化します
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values))  # トレーニングデータのテキストをスライスしてデータセットを作成
ds = ds.map(preprocess_fn)  # 前処理関数を適用します
ds = ds.batch(16)  # バッチサイズを16に設定してデータをバッチ化します

In [ ]:
from tqdm import tqdm

# 予測結果を保存するリストを初期化します
preds = []

# データセットに対して予測を行います
for inputs in tqdm(ds):  # 進行状況を表示しながらデータセットをイテレート
    keras.backend.clear_session(free_memory=True)  # セッションをクリアしてメモリを解放します
    preds.append(model(inputs))  # モデルを使用して入力データから予測を行い、結果をリストに追加します
    keras.backend.clear_session()  # 再びセッションをクリアしてメモリを解放します

In [ ]:
import numpy as np

# 予測結果を連結します
results = np.concatenate(preds)  # リストに格納された予測結果を一つの配列にまとめます

In [ ]:
import pandas

# 予測結果をDataFrameに変換します
submission = pandas.DataFrame(data=results, index=raw_test_dataset.id, columns=label_columns)  # 結果をDataFrameに格納し、インデックスをテストデータセットのIDに設定します

In [ ]:
# 提出用ファイルをCSV形式で保存します
submission.to_csv('submission.csv', index=False)  # インデックスを含めずに'submission.csv'として保存します

In [ ]:
# 提出用DataFrameの最初の5行を表示します
submission.head()  # DataFrameの冒頭の内容を確認して、予測結果が正しく保存されているか確認します

In [ ]:
# このセルにはコードが含まれていません。必要な処理やデータをここに追加してください。

---

# コメント

> ## ano
> 
> ノートブックのバージョン18を2回提出しました。どちらもタイムアウトによるエラー提出となりました。コードを改善してスピードを上げる必要があります。とにかく、素晴らしいスタートですね！
> 
> 
> 
> > ## Pranshu Bahadur（トピック著者）
> > 
> > そうですね…バージョン19がタイムアウトしないことを願っています。スピードアップのために何か提案はありますか？
> > 
> > 
> > 
> > > ## ano
> > > 
> > > 一番最初に思い浮かんだのは、[このノートブック](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora?scriptVersionId=187740026)のような並列計算です。cuda 0のモデルとcuda 1のモデルを使い、スレッドプールを使って同時に推論します。しかし、100%確実とは言えません。コードを書くことはできますが、正直言って私はLLMやkeras_nlpなどに新しいです…
> > > 
> > > 
> > > 
> > > ## Pranshu Bahadur（トピック著者）
> > > 
> > > スレッドプールに関して調べてみます。何らかの形で機能するかもしれません。正直、私もLLMについてはとても新しいですが、実装を通じて学んでいます！
> > > 
> > > 

---

> ## carvingfate
> 
> 素晴らしい仕事です！Gemma-2は最近のバージョンのトランスフォーマーが必要で、これが原因で古いクラウドGPUプラットフォームでのトレーニング中に多くの問題が発生しました。コードを共有してくれてありがとう。
> 
> 

---

> ## Lorry Zou
> 
> 今日は土曜日です。提出の成功はありましたか？
> 
> 
> 

---